In [1]:
import pandas as pd

In [2]:
# Data from 
# https://en.wikipedia.org/wiki/List_of_members_of_the_Quorum_of_the_Twelve_Apostles_(LDS_Church)
# Extracted using
# https://wikitable2csv.ggor.de/

path = '../data/'
filedata = {
    'original': 'apostles_orig12.csv',
    'non-quorum': 'apostles_non_quorum.csv',
    '': 'apostles_others.csv'}

results = []
for filestr, filename in filedata.items():
    df_ = pd.read_csv(path+filename, names=['empty', 'key', 'val', 'val2'])
    df_['idx'] = (df_['key']=='Name:').cumsum()
    df = df_.pivot(columns='key', values='val', index='idx')
    df.columns = df.columns.str.strip(':')
    for col in ['Born', 'Died']:
        df[col] = df[col].str.extract('([0-9]*-[0-9]*-[0-9]*)')
    df['Category'] = filestr
    results.append(df)

In [3]:
# collect
collected_results = pd.concat(results)[['Name', 'Born', 'Died', 'Positions', 'Notes', 'Category']]
collected_results = collected_results.reset_index(drop=True)
collected_results['Positions'] = collected_results['Positions'].str.replace(
    '[^ ()–]*(\([0-9]*-[0-9]*-[0-9T]*\))', '\\1', regex=True)

In [4]:
# The positions are separated (mostly) by line feeds so split them and clean up the results
pos_data = collected_results['Positions'].str.replace(
    '\n(', ' (', regex=False).str.split('\n').explode().str.split(',').to_frame('pos_data')
pos_data['position'] = pos_data['pos_data'].str[0].str.strip(' ')
pos_data['called_by'] = pos_data['pos_data'].str[1].str.strip(' ')
pos_data['dates'] = pos_data['pos_data'].str[2].str.strip(' ')
cb_missing = ~pos_data['called_by'].str.startswith('called').astype('bool')
pos_data.loc[cb_missing, 'dates'] = pos_data.loc[cb_missing, 'called_by']
pos_data.loc[cb_missing, 'called_by'] = ''
pos_data['called_by'] = pos_data['called_by'].str.lstrip('called by ')

# clean up certain titles with variations
cleanup_list = (
    ('[0-9].*President of [Tt]he Church.*', 'President of the Church'),
    ('.*Presiden.* of .* Seventy', 'Presidency of the Seventy'),
    ('^Apostle$', 'LDS Church Apostle'),
    (' \(with.*\)', ''),
    ('First Council of the Seventy', 'First Quorum of the Seventy'), 
    ('Second Counselor in First Presidency', 'Second Counselor in the First Presidency'))
for a, b in cleanup_list:
    pos_data['position'] = pos_data['position'].str.replace(a, b, regex=True)

pos_data['date_start'] = pos_data['dates'].str.extract('([0-9]{4}-[0-9]{2}-[0-9]{2})')
pos_data['date_end'] = pos_data['dates'].str.extract('–.*([0-9]{4}-[0-9]{2}-[0-9]{2})')
pos_data_final = pos_data.drop(columns=['pos_data', 'dates'])

In [5]:
calling_data = collected_results[['Name']].join(pos_data_final)

In [6]:
# Add extra data for the ones who ran church universities
other_data = {
    'Name': ['Dallin H. Oaks', 'Jeffrey R. Holland', 'Henry B. Eyring', 'David A. Bednar'],
    'position': ['[non-GA] President of ' + val for val in ['BYU-Provo', 'BYU-Provo', 'Ricks College', 'BYU-Idaho']],
    'called_by': ['Joseph Fielding Smith', 'Spencer W. Kimball', 'Joseph Fielding Smith', 'Gordon B. Hinckley'],
    'date_start': ['1971-08-01', '1980-09-01', '1971-08-01', '1997-07-01'],
    'date_end': ['1980-08-01', '1989-03-31', '1977-08-01', '2004-10-01']}

calling_data_plus = pd.concat([calling_data,
                               pd.DataFrame(other_data,
                                            index=calling_data[calling_data['Name'].isin(
                                                other_data['Name'])]['Name'].drop_duplicates().index)])

In [7]:
calling_data_plus.sort_index().to_csv(path + 'apostles_calling_data.csv')

In [8]:
bio_data = collected_results[['Name', 'Born', 'Died', 'Notes', 'Category']]
bio_data.to_csv(path + 'apostles_bio_data.csv')